In [1]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support
from datasets import Dataset

# Load dataset
file_path = '/kaggle/input/made-up-dataset/train_made_up_dataset.csv'
df = pd.read_csv(file_path)
# Drop the unnecessary column and encode the labels
df = df[['Code', 'Smell']]
label_encoder = LabelEncoder()
df['Smell'] = label_encoder.fit_transform(df['Smell'])

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare the datasets for Hugging Face's Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenizer and model setup
model_name = 'microsoft/graphcodebert-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder.classes_))

# Tokenization function
def tokenize_function(examples):
    encoding = tokenizer(examples['Code'], truncation=True, padding='max_length', max_length=512)
    encoding['labels'] = examples['Smell']
    return encoding

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# Define metrics for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    return {'f1_macro': f1}

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    load_best_model_at_end=True,
    metric_for_best_model='f1_macro'
)

# Trainer 0e92fed1c003f0fdabc054357fbb76beb18ff2dc
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the best model
trainer.save_model('./best_model')


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

...

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1 Macro
1,No log,0.942162,0.918258
2,No log,0.331400,0.959383
3,No log,0.153168,0.965855
4,No log,0.113217,0.965855
5,No log,0.152700,0.965855
6,No log,0.154058,0.965855
7,No log,0.140725,0.965855
8,No log,0.143277,0.965855
9,No log,0.146906,0.965855
10,No log,0.147586,0.965855


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

In [2]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from datasets import Dataset


test_df = pd.read_csv("/kaggle/input/made-up-dataset/test_made_up_dataset.csv")
test_df.dropna(inplace=True)
# Prepare the test dataset for Hugging Face's Dataset format
test_dataset = Dataset.from_pandas(test_df)
# Ensure the 'smell' column exists in the test DataFrame
if 'Smell' not in test_df.columns:
    raise ValueError("The test DataFrame does not contain the 'smell' column, which is required for evaluation.")

# Tokenization function with labels included
def tokenize_function(examples):
    encoding = tokenizer(examples['Code'], truncation=True, padding='max_length', max_length=512)
    encoding['labels'] = examples['Smell']  # Assign labels during tokenization
    return encoding

# Tokenize the test dataset
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [3]:

# Re-run the evaluation process
predictions = trainer.predict(tokenized_test_dataset)
preds = predictions.predictions.argmax(axis=-1)
test_df['Smell'] = label_encoder.transform(test_df['Smell'])

labels = test_df['Smell'].values

# Validate if labels are still None
if labels is None:
    print("Labels are still None, indicating an issue during tokenization.")
else:
    # Calculate metrics if labels are found
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1_macro, _ = precision_recall_fscore_support(labels, preds, average='macro')
    mcc = matthews_corrcoef(labels, preds)

    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Macro: {f1_macro:.4f}")
    print(f"MCC: {mcc:.4f}")



/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):


Accuracy: 0.9792
Precision: 0.9815
Recall: 0.9792
F1 Macro: 0.9791
MCC: 0.9755
